In [1]:
import numpy as np
import astropy.units as u
from astropy.time import Time
from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.twobody.propagation import cowell

# Constants
mu = Earth.k.decompose().value  # Earth's mu in km^3/s^2 (~398600.4418)
R_earth = Earth.R.to(u.km).value     # Earth radius ~6371 km
R_earth_m = R_earth * 1000          # Earth radius in meters

# Atmosphere model parameters
rho0 = 1.225         # kg/m^3 at sea level
H = 8500.0           # scale height in meters

# Spacecraft parameters
Cd = 1.0             # Drag coefficient
A = 4.15             # Cross-sectional area in m^2
mass = 2460.0        # spacecraft mass in kg

def atmosphere_density(alt_m):
    # alt_m in meters above Earth's surface
    # Simple exponential model
    return rho0 * np.exp(-alt_m / H)

def drag_acceleration(t0, state, k):
    # state = [rx, ry, rz, vx, vy, vz] in [km, km/s]
    r_vec = state[:3]
    v_vec = state[3:]

    r_norm = np.linalg.norm(r_vec)
    # Altitude in meters: altitude = r_norm (km) * 1000 - R_earth_m
    alt_m = (r_norm * 1000) - R_earth_m
    
    if alt_m < 0:
        # Below Earth's surface (simulation end condition),
        # but let's continue to apply drag anyway.
        alt_m = 0

    rho = atmosphere_density(alt_m)  # kg/m^3

    v_norm = np.linalg.norm(v_vec)
    if v_norm == 0:
        return -k * r_vec / r_norm**3  # Just gravity if no velocity

    # Convert velocity to m/s from km/s
    v_m_s = v_norm * 1000.0
    v_hat = v_vec / v_norm

    # Drag force per unit mass (in m/s^2):
    # a_drag = -0.5 * (Cd * A / mass) * rho * v^2 * direction
    a_drag_m_s2 = -0.5 * (Cd * A / mass) * rho * v_m_s**2 * v_hat

    # Convert a_drag to km/s^2
    a_drag_km_s2 = a_drag_m_s2 / 1000.0

    # Gravitational acceleration in km/s^2
    a_grav = -k * r_vec / r_norm**3

    return a_grav + a_drag_km_s2

# Initial orbit: ~200 km altitude circular orbit
altitude = 200.0 * u.km
r0 = R_earth + altitude.value
# Circular orbit velocity
v_circ = np.sqrt(mu / r0)

# Initial state
r0_vec = np.array([r0, 0, 0])       # km
v0_vec = np.array([0, v_circ, 0])   # km/s

state0 = np.hstack((r0_vec, v0_vec))

# Time span: propagate for a few orbits or until reentry
t_span = (0 * u.s, 20000 * u.s)  # about ~5.5 hours

# Integrate with Cowell
solution = cowell(
    Earth.k,
    state0,
    t_span,
    f=drag_acceleration,  # our custom acceleration function including drag
    rtol=1e-8,
    atol=1e-8
)

# Extract results
times = solution.t
states = np.array(solution.y).T  # shape (N,6)

# Post-processing
altitudes = np.linalg.norm(states[:, :3], axis=1) - R_earth

# Print some final results
print("Final altitude:", altitudes[-1], "km")
print("Minimum altitude reached:", np.min(altitudes), "km")

# If you have matplotlib, you can plot altitude over time:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(times.to(u.min), altitudes)
plt.xlabel("Time [min]")
plt.ylabel("Altitude [km]")
plt.title("Vostok Capsule Reentry Trajectory (Simplified)")
plt.grid(True)
plt.show()


c:\Users\jacki\miniforge-pypy3\envs\python_3.9\lib\site-packages\poliastro\core\_jit.py:29: UserWarning: Could not import numba package. All poliastro functions will work properly but the CPU intensive algorithms will be slow. Consider installing numba to boost performance.
  warnings.warn(


TypeError: cowell() missing 1 required positional argument: 'tofs'

In [2]:
import numpy as np
import astropy.units as u
from astropy import constants as const
import matplotlib.pyplot as plt

from poliastro.bodies import Earth
from poliastro.twobody.orbit import Orbit
from poliastro.twobody.propagation import cowell

# Earth's parameters
mu = Earth.k.decompose().value  # Earth's GM, in km^3/s^2
R_earth = Earth.R.to(u.km).value  # Earth radius in km
R_earth_m = R_earth * 1000.0

# Atmosphere model parameters (simple exponential)
rho0 = 1.225       # kg/m^3 at sea level
H = 8500.0         # scale height in m

def atmosphere_density(alt_m):
    # alt_m in meters
    return rho0 * np.exp(-alt_m / H)

# Spacecraft parameters (approx Vostok)
Cd = 1.0          # Drag coefficient (approx)
A = 4.15          # Cross-sectional area in m^2
mass = 2460.0      # spacecraft mass in kg

def drag_accel(t0, state, k):
    # state = [rx, ry, rz, vx, vy, vz] in [km, km/s]
    r_vec = state[:3]
    v_vec = state[3:]

    r_norm = np.linalg.norm(r_vec)
    alt_m = (r_norm * 1000.0) - R_earth_m
    if alt_m < 0:
        alt_m = 0.0  # If below Earth surface, clamp at 0 for density calc

    rho = atmosphere_density(alt_m)

    v_norm = np.linalg.norm(v_vec)
    if v_norm == 0:
        # Just return gravitational acceleration if no velocity
        return -k * r_vec / r_norm**3

    # Convert velocity to m/s
    v_m_s = v_norm * 1000.0
    v_hat = v_vec / v_norm

    # Drag acceleration in m/s^2
    a_drag_m_s2 = -0.5 * (Cd * A / mass) * rho * v_m_s**2 * v_hat
    # Convert to km/s^2
    a_drag_km_s2 = a_drag_m_s2 / 1000.0

    # Gravitational acceleration in km/s^2
    a_grav = -k * r_vec / r_norm**3

    return a_grav + a_drag_km_s2

# Initial conditions: ~200 km altitude circular LEO
altitude = 200.0 * u.km
r0 = R_earth + altitude.value
v_circ = np.sqrt(mu / r0)

r0_vec = np.array([r0, 0, 0])  # km
v0_vec = np.array([0, v_circ, 0])  # km/s

# Define times of flight
tofs = np.linspace(0, 20000, 500)*u.s  # 0 to ~5.5 hours in 500 steps

# Call cowell with custom acceleration
r, v = cowell(mu, r0_vec, v0_vec, tofs, f=drag_accel, rtol=1e-8, atol=1e-8)

# Extract altitudes
r_norms = np.linalg.norm(r, axis=1)
altitudes = r_norms - R_earth

print("Final altitude:", altitudes[-1], "km")
print("Minimum altitude reached:", np.min(altitudes), "km")

# Plot altitude vs time
plt.figure()
plt.plot(tofs.to(u.min), altitudes)
plt.xlabel("Time [min]")
plt.ylabel("Altitude [km]")
plt.title("Vostok Capsule Reentry (Simplified)")
plt.grid(True)
plt.show()


AttributeError: 'numpy.float64' object has no attribute 'to'

: 